In [ ]:
from pymongo import MongoClient
import requests as rq
import datetime as dt
import time

cardiac = [
    'http://nosql.hpeixoto.me/api/sensor/3001',
    'http://nosql.hpeixoto.me/api/sensor/3002',
    'http://nosql.hpeixoto.me/api/sensor/3003',
    'http://nosql.hpeixoto.me/api/sensor/3004',
    'http://nosql.hpeixoto.me/api/sensor/3005'
]

blood = [
    'http://nosql.hpeixoto.me/api/sensor/4001',
    'http://nosql.hpeixoto.me/api/sensor/4002',
    'http://nosql.hpeixoto.me/api/sensor/4003',
    'http://nosql.hpeixoto.me/api/sensor/4004',
    'http://nosql.hpeixoto.me/api/sensor/4005'
]

url = "mongodb://localhost:27017"
client = MongoClient(url)
db = client.sensors

# Only the first result matters
def getLastTimestamp(col):
    m = col.find({},{"_id":0, "timestamp": 1}).sort("_id",-1).limit(1)
    try: 
        return dt.datetime.strptime((m.next())['timestamp'], "%Y-%m-%d %H:%M:%S")
    except:
        return dt.datetime(1,1,1)

# Only insert if the timestamp is more recent than the last on the db
def maybeInsert(col,data,tsdb,tsjson):
    if(tsdb >= tsjson):
        return False
    else:
        col.insert_one(data)
        return True

# Insert into db = sensors, collection = sensorYYYY
def parseJson(data,typ):
    print(">> Currently on sensor: type = {0}, ID = {1}".format(typ, data['sensorid']))
    id = str(data['sensorid'])
    col = db[typ + id]
    date1 = getLastTimestamp(col)
    date2 = dt.datetime.strptime(data['timestamp'], "%Y-%m-%d %H:%M:%S")
    result = maybeInsert(col,data,date1,date2) 
    print(str(date2) + ' > ' + str(date1) + ' ?', "Yes, adding the sensor data to the DB.\n" if result else "No, skipping the current sensor data.\n")

while True:
    print("New Round of GET Requests...")
    for sensor in cardiac:
        data = rq.get(sensor).json()
        parseJson(data, 'cardiac')

    for sensor in blood:
        data = rq.get(sensor).json()
        parseJson(data, 'blood')
    
    time.sleep(5)
